In [1]:
import os
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np

import depth_controller

np.set_printoptions(precision=2, suppress=True)

Segmentation masks:

0: background

1: tool

2: ILM

3: RPE


In [ ]:
masks_path = '/home/demir/Desktop/jhu_project/oct-depth-control/data/may21-1/masks_machine'
masks = []
for mask_img in os.listdir(masks_path):
    mask = cv2.imread(os.path.join(masks_path, mask_img), cv2.IMREAD_GRAYSCALE)
    masks.append(mask)

In [ ]:
test_img = cv2.imread('/home/demir/Desktop/jhu_project/oct-depth-control/data/may21-1/masks_machine/volume_0_slice_2.png', cv2.IMREAD_GRAYSCALE)
print(np.unique(test_img))
filtered_img = (test_img == 1).astype(int)
plt.imshow(test_img)

In [ ]:
# example usage
# target depth should be selected by the user in the beginning
target_depth = 0.5
# Segmented b-scans should come from the network as a list
segmented_b_scans = [] # each b-scan is a 2D numpy array
slice_idx, needle_tip = depth_controller.volume_find_needle_tip_largest_component(segmented_b_scans) 
a_scan, ilm_row, rpe_row = depth_controller.find_layers_coords(segmented_b_scans[slice_idx], needle_tip[0])
normalized_pos = depth_controller.calc_normalized_pos_between_layers(ilm_row, rpe_row, needle_tip[1])
if normalized_pos >= target_depth:
    print('needle tip is at or below target depth')

### Test using real segmentation results

In [3]:
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]  

In [6]:
segmented_images_path = '/home/demir/Desktop/jhu_project/needle-segmentation/test_seg_res'
oct_images_path = '/home/demir/Desktop/jhu_project/oct_scans/jun11/2.1/images'

num_b_scans_volume = 5
target_depth = 0.5

seg_masks = [cv2.imread(os.path.join(segmented_images_path, img), cv2.IMREAD_UNCHANGED) for img in sorted(os.listdir(segmented_images_path), key=natural_sort_key)]
oct_images = [cv2.imread(os.path.join(oct_images_path, img)) for img in sorted(os.listdir(oct_images_path), key=natural_sort_key)]

for i in range(len(seg_masks)//num_b_scans_volume):
    volume_seg_masks = seg_masks[i*num_b_scans_volume:(i+1)*num_b_scans_volume]
    volume_oct_images = oct_images[i*num_b_scans_volume:(i+1)*num_b_scans_volume]
    slice_idx, needle_tip = depth_controller.volume_find_needle_tip_largest_component(volume_seg_masks)
    a_scan, ilm_row, rpe_row = depth_controller.find_layers_coords(volume_seg_masks[slice_idx], needle_tip[0])
    normalized_pos = depth_controller.calc_normalized_pos_between_layers(ilm_row, rpe_row, needle_tip[1])
    if normalized_pos >= target_depth:
        print('needle tip is at or below target depth', normalized_pos)
    plt.imshow(volume_oct_images[slice_idx])
    plt.imshow(volume_seg_masks[slice_idx], alpha=0.5)
    plt.scatter(needle_tip[1], needle_tip[0], c='r')
    plt.show()

current a scan: 640
current a scan: 639
current a scan: 638
current a scan: 637
current a scan: 636
current a scan: 635
current a scan: 634
current a scan: 633
current a scan: 632
current a scan: 631
current a scan: 630
current a scan: 629
current a scan: 628
current a scan: 627
current a scan: 626
current a scan: 625
current a scan: 624
current a scan: 623
current a scan: 622
current a scan: 621
current a scan: 620
current a scan: 619
current a scan: 618
current a scan: 617
current a scan: 616
current a scan: 615
current a scan: 614
current a scan: 613
current a scan: 612
current a scan: 611
current a scan: 610
current a scan: 609
current a scan: 608
current a scan: 607
current a scan: 606
current a scan: 605
current a scan: 604
current a scan: 603
current a scan: 602
current a scan: 601
current a scan: 600
current a scan: 599
current a scan: 598
current a scan: 597
current a scan: 596
current a scan: 595
current a scan: 594
current a scan: 593
current a scan: 592
current a scan: 591



KeyboardInterrupt

